# Sourcing the Wheat Price Data
In this notebook, we will be sourcing the wheat price data from the IMF api.

In [1]:
# Importing the necessary libraries
import sdmx

In [2]:
# Retrieving test data
IMF_DATA = sdmx.Client('IMF_DATA')
data_msg = IMF_DATA.data('CPI', key = 'USA+CAN.CPI.CP01.IX.M', params = {'startPeriod' : 2018})

# To pandas
cpi_df = sdmx.to_pandas(data_msg)

# Preview
cpi_df.head()

xml.Reader got no structure=… argument for StructureSpecificData


COUNTRY  INDEX_TYPE  COICOP_1999  TYPE_OF_TRANSFORMATION  FREQUENCY  COMMON_REFERENCE_PERIOD  OVERLAP  SCALE  ACCESS_SHARING_LEVEL  SECURITY_CLASSIFICATION  TIME_PERIOD
CAN      CPI         CP01         IX                      M          2002A                    OL       0      PUBLIC_OPEN           PUB                      2018-M01       142.6
                                                                                                                                                             2018-M02       142.3
                                                                                                                                                             2018-M03       141.3
                                                                                                                                                             2018-M04       141.9
                                                                                                                       

In [3]:
# Checking for the correct Dataset
IMF_DATA.dataflow()

<sdmx.StructureMessage>
  <Header>
    id: 'IDREF7533'
    prepared: '2025-07-28T09:31:04.029924+00:00'
    sender: <Agency unknown>
    source: 
    test: False
  response: <Response [200]>
  DataflowDefinition (65): MCDREO HPD CTOT ISORA_2018_DATA_PUB DIP AFRR...
  DataStructureDefinition (57): IMF.MCD:DSD_MCDREO(7.0.0) IMF.FAD:DSD_H...

In [4]:
# Confirming the PCPS_TABLE dataflow exists
flows_msg = IMF_DATA.dataflow()

flows = flows_msg.dataflow

for flow_id, flow in flows.items():
    if 'commodity' in flow.name.en.lower() or 'price' in flow.name.en.lower():
        print(flow_id, ':', flow.name.en)

MFS_FMP : Monetary and Financial Statistics (MFS_FMP): Financial Market Prices
PCPS : Primary Commodity Price System (PCPS)
CPI : Consumer Price Index (CPI)
PPI : Producer Price Index (PPI)
CTOT : Commodity Terms of Trade (CTOT)


In [5]:
# Getting the structure ID from the PCPS dataflow
pcps_flow = flows_msg.dataflow['PCPS']

# Checking the structure reference
print(pcps_flow.structure.id)

DSD_PCPS


In [6]:
# Getting the structure message
structure_msg = IMF_DATA.get(
    resource_type = 'datastructure',
    resource_id = 'DSD_PCPS'
)

# Getting the datastructure
dsd = list(structure_msg.structure.values())[0]

# Getting the dimensions
dims = list(dsd.dimensions)

print('Dimensions for PCPS:')
for d in dims:
    print('-', d.id)

Dimensions for PCPS:
- COUNTRY
- INDICATOR
- DATA_TRANSFORMATION
- FREQUENCY
- TIME_PERIOD


In [7]:
# Grab the actual codelists
codelists = structure_msg.codelist

# Find the one for INDICATOR
for cl_id, cl in codelists.items():
    if 'INDICATOR' in cl_id:
        print(f"\nCodelist: {cl_id}")
        for code in cl:
            if 'wheat' in code.name.en.lower():
                print(code.id, ":", code.name.en)


Codelist: CL_PCPS_INDICATOR
PWHEAMT : Wheat, US dollars per metric tonne, Unit prices


In [8]:
# Fetching the wheat price data
wheat_msg = IMF_DATA.data(
    resource_id='PCPS',
    key='.PWHEAMT.INDEX.M',
    params={'startPeriod': 2015}
)

# Creating a pandas DataFrame
wheat_df = sdmx.to_pandas(wheat_msg)

# Preview
wheat_df.head()

xml.Reader got no structure=… argument for StructureSpecificData


COUNTRY  INDICATOR  DATA_TRANSFORMATION  FREQUENCY  SCALE  TIME_PERIOD
G001     PWHEAMT    INDEX                M          0      2015-M01       148.756962
                                                           2015-M02       140.880382
                                                           2015-M03       141.543337
                                                           2015-M04       136.822560
                                                           2015-M05       134.926862
Name: value, dtype: float64

In [9]:
wheat_df

COUNTRY  INDICATOR  DATA_TRANSFORMATION  FREQUENCY  SCALE  TIME_PERIOD
G001     PWHEAMT    INDEX                M          0      2015-M01       148.756962
                                                           2015-M02       140.880382
                                                           2015-M03       141.543337
                                                           2015-M04       136.822560
                                                           2015-M05       134.926862
                                                                             ...    
                                                           2025-M01       133.120360
                                                           2025-M02       132.752345
                                                           2025-M03       125.422716
                                                           2025-M04       122.077907
                                                           2025-M05       137.4